In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

print(f"TensorFlow version: {tf.__version__}")


# Classificação de Gatos e Cachorros com CNN

Este notebook implementa um modelo de classificação de imagens usando TensorFlow/Keras para distinguir entre gatos e cachorros.


In [ ]:
# Definir caminhos do dataset
base_path = 'farmtech_yolo_project/dataset'
train_path = os.path.join(base_path, 'train/images')
test_path = os.path.join(base_path, 'test/images')
valid_path = os.path.join(base_path, 'valid/images')

# Verificar se os diretórios existem
print(f"Train path exists: {os.path.exists(train_path)}")
print(f"Test path exists: {os.path.exists(test_path)}")
print(f"Valid path exists: {os.path.exists(valid_path)}")


In [ ]:
# Função para carregar imagens e labels
def load_images_and_labels(images_path, img_size=(128, 128)):
    """
    Carrega imagens do diretório e cria labels baseado no nome do arquivo
    cat.*.jpg -> label 0
    dog.*.jpg -> label 1
    """
    images = []
    labels = []
    
    # Listar todos os arquivos no diretório
    files = sorted(os.listdir(images_path))
    
    for filename in files:
        if filename.endswith('.jpg'):
            # Carregar e processar a imagem
            img_path = os.path.join(images_path, filename)
            img = Image.open(img_path)
            img = img.resize(img_size)
            img_array = np.array(img)
            
            # Garantir que a imagem está em RGB (3 canais)
            if len(img_array.shape) == 2:  # Se for grayscale
                img_array = np.stack([img_array] * 3, axis=-1)
            
            images.append(img_array)
            
            # Criar label baseado no nome do arquivo
            if filename.startswith('cat'):
                labels.append(0)  # Gato = 0
            elif filename.startswith('dog'):
                labels.append(1)  # Cachorro = 1
    
    return np.array(images), np.array(labels)

print("Função de carregamento criada!")


In [ ]:
# Carregar o dataset
print("Carregando dataset...")

train_images, train_labels = load_images_and_labels(train_path)
test_images, test_labels = load_images_and_labels(test_path)
valid_images, valid_labels = load_images_and_labels(valid_path)

print()
print(f"Train images shape: {train_images.shape}")
print(f"Train labels: {len(train_labels)}")
print(f"Train labels content: {train_labels}")
print()
print(f"Test images shape: {test_images.shape}")
print(f"Test labels: {len(test_labels)}")
print()
print(f"Valid images shape: {valid_images.shape}")
print(f"Valid labels: {len(valid_labels)}")


In [ ]:
# Salva as classes
class_names = ['Cat', 'Dog']
print(f"Classes: {class_names}")


In [ ]:
# Exibir algumas imagens do dataset de treino
plt.figure(figsize=(12, 8))
num_images = min(20, len(train_images))
for i in range(num_images):
    plt.subplot(4, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i]])
plt.tight_layout()
plt.show()


In [ ]:
# Normalizar os dados
# Toda imagem é representada por uma matriz de pixels que vai de 0 a 255
# Redes neurais funcionam melhor com valores de 0 a 1 ou -1 a 1.
# Então vamos dividir os valores da matriz por 255 para ter uma escala de 0 a 1
train_images = train_images / 255.0
test_images = test_images / 255.0
valid_images = valid_images / 255.0

print("Imagens normalizadas!")
print(f"Train images range: [{train_images.min():.2f}, {train_images.max():.2f}]")


## Modelo utilizando TensorFlow/Keras


In [ ]:
# Definição da semente de aleatoriedade
tf.random.set_seed(42)

# Define o modelo CNN (Convolutional Neural Network)
model = keras.Sequential([
    # Camada de entrada
    keras.layers.Input(shape=(128, 128, 3)),
    
    # Primeira camada convolucional
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    # Segunda camada convolucional
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    # Terceira camada convolucional
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    # Achatamento
    keras.layers.Flatten(),
    
    # Camadas densas
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout para evitar overfitting
    
    # Camada de saída (2 classes: cat, dog)
    keras.layers.Dense(2, activation='softmax')
])

# Compilar o modelo
model.compile(
    optimizer='adam',  # Algoritmo de otimização
    loss='sparse_categorical_crossentropy',  # Função de perda para classificação
    metrics=['accuracy']  # Métrica para avaliar
)

print("Modelo criado e compilado!")


In [ ]:
# O que tem neste modelo? Exibir a arquitetura do modelo
model.summary()


In [ ]:
# Treinamento do modelo
print("Iniciando treinamento...")
history = model.fit(
    train_images, 
    train_labels, 
    epochs=20,
    validation_data=(valid_images, valid_labels),
    batch_size=8
)
print("Treinamento concluído!")


In [ ]:
# Avaliação do modelo com o conjunto de teste
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print()
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:
# Visualizar histórico de treinamento
plt.figure(figsize=(12, 4))

# Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Acurácia durante o Treinamento')
plt.grid(True)

# Perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Perda durante o Treinamento')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Fazer predições em algumas imagens de teste
predictions = model.predict(test_images)

# Visualizar predições
plt.figure(figsize=(12, 8))
num_images = min(8, len(test_images))
for i in range(num_images):
    plt.subplot(2, 4, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i])
    
    predicted_label = np.argmax(predictions[i])
    true_label = test_labels[i]
    
    # Cor verde se correto, vermelho se errado
    color = 'green' if predicted_label == true_label else 'red'
    
    plt.xlabel(f"Pred: {class_names[predicted_label]}\nTrue: {class_names[true_label]}", 
               color=color)

plt.tight_layout()
plt.show()


In [ ]:
# Salvar o modelo treinado
model_save_path = 'cats_dogs_model.keras'
model.save(model_save_path)
print(f"Modelo salvo em: {model_save_path}")


## Resumo do Modelo

Este notebook implementou um modelo CNN (Convolutional Neural Network) para classificação de imagens de gatos e cachorros usando TensorFlow/Keras.

### Arquitetura do Modelo:
- **Camadas Convolucionais**: 3 camadas conv2D com filtros 32, 64 e 128
- **Pooling**: MaxPooling após cada convolução para redução dimensional
- **Camada Densa**: 128 neurônios com ativação ReLU
- **Dropout**: 0.5 para regularização
- **Saída**: 2 neurônios (cat/dog) com ativação softmax

### Dataset:
- **Training**: 64 imagens (32 gatos + 32 cachorros)
- **Validation**: 8 imagens (4 gatos + 4 cachorros)
- **Test**: 8 imagens (4 gatos + 4 cachorros)

### Pré-processamento:
- Redimensionamento para 128x128 pixels
- Normalização (valores entre 0 e 1)
- Conversão para RGB (3 canais)

### Parâmetros de Treinamento:
- **Epochs**: 20
- **Batch Size**: 8
- **Optimizer**: Adam
- **Loss Function**: Sparse Categorical Crossentropy
